All functions in this cell have been taken from the PHYS2030 _Colabs_ code produced by Prof. Michael Steel (School of Mathematical & Physical Sciences, Macquarie University). I don't know how important any of it is to the Wigner evolution code, but I'm too scared to touch it for legacy reasons.

In [ ]:
m_e = 9.10938356e-31; m_p = 1.6726219e-27; m_n = 1.674927471e-27; q_e = 1.60217662e-19; h_planck = 6.62607015e-34
hbar = 1.0545718176e-34; k_boltzmann = 1.380649e-23; c_vac = 299792458; epsilon_0 = 8.8541878128e-12 ;mu_0 = 1.25663706212e-6
G_newton = 6.67408e-11; a_Bohr = 5.29177228e-11


from math import *
import numpy as np
from functools import wraps
import numpy.random as nprand
import numpy.linalg as linalg
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as op
import matplotlib.cm as cm
import scipy.linalg as spla
import scipy.integrate
import scipy.interpolate
from scipy.linalg.decomp import eigvalsh_tridiagonal
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D  # Needed for 3d plot
from IPython.display import display, HTML, Video, Image
import copy
mpl.rcParams['animation.embed_limit'] = 2**128

def ensure_axes(close=False, fig_kwargs=None, ax_kwargs=None):

    """
    This function decorator ensures that an axes object with the correct settings is given, or created if needed.

    Parameters
    ----------
    close: bool
        whether to return the figure and axes objects or close them (preventing double display)
    fig_kwargs: dict
        additional arguments for figure creation
    ax_kwargs: dict
        additional arguments for axes creation
    """

    if fig_kwargs is None:
        fig_kwargs = {}
    if ax_kwargs is None:
        ax_kwargs = {}

    def decorator(func):
        @wraps(func)
        def wrapper(*args, ax=None, **kwargs):
            projection = ax_kwargs.get("projection")
            if projection is not None:
                if ax is not None and ax.name != projection:
                    raise ValueError((
                        "The axes object passed must use the {}"
                        "projection"
                    ).format(projection))
            if ax is None:
                fig, ax = plt.subplots(
                    subplot_kw=ax_kwargs, **fig_kwargs
                )
            else:
                fig = ax.get_figure()
            func(*args, ax=ax, **kwargs)
            if not close:
                return fig, ax
            plt.close(fig)
            return None
        return wrapper
    return decorator

def control_animation(func):

    """
    A function decorator which takes the FuncAnimation object returned by a function and displays it correctly in a jupyter notebook. This function
    also adds an additional argument to control which backend to use to display the animation.
    """

    @wraps(func)
    def wrapper(*args, animation_method='html5', **kwargs):
        anim = func(*args, **kwargs)
        animation_method = animation_method.lower()
        if animation_method == 'html5':
            display(HTML(anim.to_html5_video()))
            plt.close()
        elif animation_method == 'jshtml':
            display(HTML(anim.to_jshtml()))
        elif animation_method == 'mp4':
            fname = mkstemp(prefix=func.__name__, suffix='.mp4')[1]
            anim.save(fname)
            display(Video(fname))
        elif animation_method == 'gif':
            fname = mkstemp(prefix=func.__name__, suffix='.gif')[1]
            anim.save(fname, writer='imagemagick')
            display(Image(fname))
        else:
            raise ValueError(
                "Unknown animation method {}".format(animation_method)
            )
        return anim
    return wrapper


def make_axes(rows, cols, dimstr=''):
    fig=plt.figure(figsize=[cols*4, rows*4])
    fig.subplots_adjust(wspace=.3, hspace=.3)
    iax=0
    axs=[]
    for r in range(rows):
        for c in range(cols):
            if len(dimstr) and dimstr[iax]=='3':
                ax=fig.add_subplot(rows, cols, iax+1, projection='3d')
            else:
                ax=fig.add_subplot(rows, cols, iax+1)
            iax+=1
            axs.append(ax)
    #return (fig, axs)
    return axs

def plot_eigenvectors(evecs, ev_nums=[1], connect_lines=True):

    '''
    Make line plots of the eigenvectors numbered in the list ev_nums extracted from the matrix of all eigenvectors evecs. Set connect_lines=False to remove the dotted lines joining the values.
    '''

    fig = plt.figure(figsize=(12,4))
    ax=fig.add_subplot(111)
    ax.set_xlabel('Basis state')
    ax.set_ylabel(r'Re$[a_j]$')

    xv=np.arange(evecs.shape[1])
    for ev in ev_nums:
        ax.plot(xv, evecs[:,ev],':', marker='.', markersize=15, label='$n={0}$'.format(ev))

    ax.legend()

def plot_all_eigenvectors(evecs, colorbar=False):

    '''
    Plot a matrix map of all eigenvectors in the matrix of eigenvectors evecs. Set colorbar=True to see the colorscale.
    '''

    fig = plt.figure(figsize=(12,4))
    ax = fig.add_subplot(131)
    ax.set_xlabel('Eig. num.')
    ax.set_ylabel(r'Re$[a_j]$')
    cax=ax.matshow(np.real(evecs),cmap=cm.get_cmap('bwr'))
    if colorbar:
        cbar = fig.colorbar(cax)

def plot_eigenvalues(l_evs, ylims=None):
    # Plot a list l_evs=[evals1, evals2, ...] of one or more arrays of eigenvalues
    # as a function of the eigenvalue index

    if not type(l_evs) is list:
        l_evs=list(l_evs)

    ax=plt.subplot(111)
    ax.set_xlabel('Eig. num.')
    ax.set_ylabel(r'$E_n$')
    if ylims is not None:ax.set_ylim(ylims)

    for t_ev in l_evs:
        sevs=t_ev
        sevs.sort()
        vindex=np.arange(len(sevs),dtype=np.int32)
        ax.plot(vindex, sevs,':',marker='.')


def plot_eigenvalues_vert(l_evs, ylims=None):
    # Plot a list l_evs=[evals1, evals2, ...] of one or more arrays of eigenvalues.
    # Each set is plotted in one vertical line with each set spaced along the x axis

    #l_evs=copy.deepcopy(evs)

    if not l_evs is list:
        l_evs=list(l_evs)

    ax=plt.subplot(111)
    ax.set_xlabel('Molecule length $n$')
    ax.set_ylabel(r'Energy $E_j$')
    if ylims is not None:ax.set_ylim(ylims)

    for t_ev in l_evs:
        sevs=t_ev
        sevs.sort()
        dim=len(sevs)
        vindex=dim+np.zeros(dim,dtype=np.int32)
        ax.plot(vindex, sevs,'.')

def plot_eigenstates(evecs, ev_nums=[1], connect_lines=True):
    # Plot selected eigenvectors and probability distributions from the matrix of eigenvectors evecs as a function of atom site number
    # The selection is specified by the list ev_nums

    fig = plt.figure(figsize=(12,4))
    ax=fig.add_subplot(121)
    ax.set_xlabel('Basis state/atom site')
    ax.set_ylabel(r'a_j$')

    xv=np.arange(evecs.shape[1])
    for ev in ev_nums:
        sign0 = evecs[:,ev][0]/abs(evecs[:,ev][0]) # pick the phase of the eigenvector so that the value at atom 0 is positive
        ax.plot(xv, evecs[:,ev]*sign0,':', marker='.', markersize=15, label='$n={0}$'.format(ev))
    ax.legend()

    ax=fig.add_subplot(122)
    ax.set_xlabel('Basis state/atom site')
    ax.set_ylabel(r'$p_j=|a_j|^2$')

    xv=np.arange(evecs.shape[1])
    for ev in ev_nums:
        ax.plot(xv, np.abs(evecs[:,ev])**2,':', marker='.', markersize=15, label='$n={0}$'.format(ev))
    ax.legend()


def plot_all_eigenstates(evecs, colorbar=False):
    # Plot all eigenvectors in matrix of eigenvectors evecs as a square matrix
    # Each eigenvector is displayed along a column with the eigenvector number increasing
    # along the x axis.
    fig = plt.figure(figsize=(12,4))
    ax = fig.add_subplot(131)
    ax.set_xlabel('Eig. num.')
    ax.set_ylabel(r'$a_j$')
    sevecs=copy.deepcopy(evecs)
    for m in range(sevecs.shape[0]): # pick the phase of each eigenvector so the atom 0 value is positive
        sevecs[:,m] = sevecs[:,m] * sevecs[:,m][0]/abs(sevecs[:,m][0])
    cax=ax.matshow(np.real(sevecs),cmap=cm.get_cmap('bwr'), origin='lower' )
    ax.xaxis.tick_bottom()
    if colorbar: cbar = fig.colorbar(cax)

def plot_potentials_1D(l_potentials, l_labels=None, xmax=50, xpts=1000,
is_radial=False, ylims=None):
    '''
    Plot one or more potential functions (V1(x), V2(x), ...) specified by l_potentials
    on the domain [-xmax, xmax] with xpts gridpoints.
    Optionally label each potential curve with a list l_label of strings.
    Use is_radial=True to restrict the domain to positive values and ylims=[ylo,yhi] to specify plotting limits.
    '''
    if not type(l_potentials) in (tuple, list): l_potentials = (l_potentials,)
    if not l_labels is None:
        if not type(l_labels) in (tuple, list): l_labels = (l_labels,)

    if is_radial:
        v_x = np.linspace(.0001, xmax, xpts)
    else:
        v_x = np.linspace(-xmax, xmax, xpts)

    fig=plt.figure(figsize=[4,4])
    ax=fig.add_subplot(111)

    for i, vfunc in enumerate(l_potentials):
        v_fx=np.zeros(xpts)
        for j,x in enumerate(v_x): v_fx[j]=vfunc(x)
        lab=None
        if not l_labels is None: lab=l_labels[i]
        ax.plot(v_x, v_fx, label=lab)
    if is_radial:
        ax.set_xlabel('r')
        ax.set_ylabel('V(r)')
    else:
        ax.set_xlabel('x')
        ax.set_ylabel('V(x)')
    if not l_labels is None: ax.legend()
    #ax.set_ylim([-100,100])
    ax.set_xlim([v_x[0],v_x[-1]])
    if not ylims is None: ax.set_ylim(ylims)

def boundstates_solve(Vfunc, xmax=50, xpts=201, numeigs=6, is_radial=False):

    '''
    Find bound state energy eigenvalues and eigenfunctions for 1D potential well problems.

    Parameters
    ----------
    domain : A tuple (x0,Xn) indicating the domain of x values on which to solve the wave equation.

    potential_func: The potential function to be solved for as created using one of the potential_builder functions.

    xpts:     Optional argument specifying the number of x-values to use.

    numeigs:  Optional argument specifying the number of eigenvalues to return.
            Typically, accuracy will be poor unless numeigs << npts.

    Returns
    -------
    A dictionary containing the following elements:
       domain  : The solution domain specified as an input paramter.
       delta_x : The spacing of x-values in the solution.
       Vx      : The potential function specified as an input paramter.
       vecx    : A numpy array of x-values.
       numeigs : The number of eigensolutions requested as an input parameter.
       evals   : A numpy array of numeigs energy eigenvalues sorted in increasing value.
       evecs   : A numpy 2D array of corresponding eigenfunctions in the same order as evals.
    '''

    # the entire solution is on the domain xlo to xhi with npts points. But psi[0] and psi[N] are fixed to zero,
    # and not part of the matrix calculation. We have to calculate on the inner (npts-1) grid points
    #xlo,xhi=domain

    if is_radial:
        vecx=np.linspace(xmax*.0001, xmax, xpts)
    else:
        vecx=np.linspace(-xmax, xmax, xpts)

    vecx_inner=vecx[1:-1]
    delta_x=vecx[1]-vecx[0]
    vp=np.array(list(map(Vfunc, vecx_inner)))
    mat=np.zeros([2, xpts-2])
    mat[0,:]=-.5/delta_x**2
    mat[1,:]=1/delta_x**2 +vp
    (evals, evecs_inner)=spla.eig_banded(mat,select='i', select_range=(0,numeigs-1)) # solve for psi_1 to psi_(N-1)

    evecs=np.zeros([xpts, numeigs], dtype=float)
    evecs[1:-1,:]=evecs_inner

    for j in range(numeigs):
        evecs[:,j]/= sqrt(scipy.integrate.simps(np.abs(evecs[:,j])**2, dx=delta_x))

    eigsolution={'vecx':vecx, 'numeigs':numeigs, 'evals':evals, 'evecs':evecs,
    'Vx': Vfunc, 'is_radial':is_radial, 'delta_x':delta_x, 'xpts':xpts}

    return eigsolution


def boundstates_plot_eigvals(soln, eiglist=None, ax=None):

    '''
    Displays a plot of the eigenvalues from a solution created by boundstates_solve.

    Parameters
    ----------
    soln    : A solution dictionary returned by a call to boundstates_solve.
              (Pass the whole dictionary, not just the 'evals' element. )

    eiglist : list of integers < numeigs
       Specifies that only the listed states should be  displayed.

    ax      : Optional axes object on which to plot for making grids of multiple plots.
    '''

    if ax is None: ax=plt.subplot(111)
    evals=soln['evals']
    eiglist = range(len(evals)) if eiglist is None else [i for i in eiglist if i < len(evals)]

    v_ev=np.zeros(len(eiglist))

    for i,e in enumerate(eiglist):
        if e >= len(evals): continue
        v_ev[i]=evals[e]
        ax.plot(e, evals[e] ,'-', marker='+', markersize=12)
    ax.plot(eiglist, v_ev , ':')

    ax.set_xlabel(r'Energy level $n$')
    ax.set_ylabel(r'$\bar{E}_n$')
    ax.set_xlim(-.25, np.array(eiglist).max()+.25)
    emin=evals[np.array(eiglist).min()]
    emax=evals[np.array(eiglist).max()]
    erange=emax-emin
    ylo=min(0, emin-erange*.05)
    yhi=emax+erange*.05

    ax.set_ylim(ylo, yhi)

def boundstates_plot_eigvals_and_well(soln, eiglist=None, ax=None, xlims=None):

    '''
    Displays a plot of the eigenvalues superimposed on the potential well from a solution created by boundstates_solve.

    Parameters
    ----------
    soln    : A solution dictionary returned by a call to boundstates_solve.
              (Pass the whole dictionary, not just the 'evals' element. )

    eiglist : list of integers < numeigs
       Specifies that only the listed states should be  displayed.

    ax      : Optional axes object on which to plot for making grids of multiple plots.
    '''

    if ax is None: ax=plt.subplot(111)
    #domain=soln['domain']
    V=soln['Vx']
    vecx=soln['vecx']
    evals=soln['evals']

    vp=np.array(list(map(V, vecx)))
    ax.plot(vecx, vp, color='dimgray')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$V(x)$')

    evals=soln['evals']
    eiglist = range(len(evals)) if eiglist is None else [i for i in eiglist if i < len(evals)]
    for i in eiglist:
        ax.plot(vecx, vecx*0+evals[i], '--')

    ylo=vp.min()

    if not xlims is None:
        ax.set_xlim(xlims[0], xlims[1])
        yhi=max(V(xlims[0]), V(xlims[1]))
    else:
        ax.set_xlim(vecx[0], vecx[-1])
        yhi=max(vp.max(), evals[np.array(eiglist).max()])

    #evs=[evals[i] for i in eiglist]
    yrange=yhi-ylo
    ax.set_ylim(ylo-yrange*.05, yhi+yrange*.05)

def boundstates_plot_eigvalsvecs_and_well(soln, eiglist=None, ax=None, xlims=None, ylims=None):

    '''
    Displays a plot of the eigenvalues and eigenfunctions superimposed on the potential well from a solution created by boundstates_solve.

    Parameters
    ----------
    soln    : A solution dictionary returned by a call to boundstates_solve.
              (Pass the whole dictionary, not just the 'evals' element. )

    eiglist : list of integers < numeigs
       Specifies that only the listed states should be  displayed.

    ax      : Optional axes object on which to plot for making grids of multiple plots.
    '''

    if ax is None: ax=plt.subplot(111)
    V=soln['Vx']
    evals=soln['evals']
    evecs=soln['evecs']

    vecx=soln['vecx']
    domain=[vecx[0], vecx[-1]]

    vp=np.array(list(map(V, vecx)))
    ax.plot(vecx, vp, color='dimgray')
    if soln['is_radial']:
      ax.set_xlabel('$r$')
      ax.set_ylabel('$V(r)$')
    else:
      ax.set_xlabel('$x$')
      ax.set_ylabel('$V(x)$')

    if xlims is None:
        ax.set_xlim(domain[0], domain[1])
    else:
        ax.set_xlim(xlims[0], xlims[1])

    eiglist = range(len(evals)) if eiglist is None else [i for i in eiglist if i < len(evals)]
    evs=[evals[i] for i in eiglist]

    ylo=vp.min()
    yhi=max(vp.max(), evals[np.array(eiglist).max()])

    if ylims is not  None:
        ylo=ylims[0]
        yhi=ylims[1]
        ax.set_ylim(ylo, yhi)
        yrange=yhi-ylo
    else:
        ax.set_ylim(ylo-yrange*.05, yhi+yrange*.05)
        yrange=yhi-ylo

    evecscale=yrange/(evecs[:,0].max())*.1
    for i in eiglist:
        ax.plot(vecx, vecx*0+evals[i], '--')
        ax.plot(vecx, evecs[:,i]*evecscale+evals[i])

def boundstates_plot_eigvecs_and_well(soln, asprobdens=False, eiglist=None, ax=None, ylims=None, xlims=None):

    '''
    Displays a plot of the eigenvectors superimposed on the potential well from a solution created by boundstates_solve.

    Parameters
    ----------
    soln    : A solution dictionary returned by a call to boundstates_solve.
              (Pass the whole dictionary, not just the 'evals' element. )

    asprobdens : boolean
       If True, causes display of probability densities rather than amplitudes.

    eiglist : list of integers < numeigs
       Specifies that only the listed states should be  displayed.

    ax      : Optional axes object on which to plot for making grids of multiple plots.
    '''

    if ax is None: ax=plt.subplot(111)

    V=soln['Vx']
    evals=soln['evals']
    evecs=soln['evecs']

    vecx=soln['vecx']
    domain=[vecx[0], vecx[-1]]
    vp=np.array(list(map(V, vecx)))

    ax.plot(vecx, vp, color='dimgray')
    if soln['is_radial']:
      ax.set_xlabel('$r$')
      ax.set_ylabel('$V(r)$')
    else:
      ax.set_xlabel('$x$')
      ax.set_ylabel('$V(x)$')

    if xlims is None:
        ax.set_xlim(domain[0], domain[1])
    else:
        ax.set_xlim(xlims[0], xlims[1])
    if ylims is None:
      ax.set_ylim(vp.min()-1, vp.max()+1)
    else:
      ax.set_ylim(ylims[0], ylims[1])
    ax2=ax.twinx()

    if asprobdens: ax2.set_ylabel('$|\psi(x)|^2$')
    else: ax2.set_ylabel('$\psi(x)$')
    ax2.yaxis.set_tick_params(labelsize=8)
    ax2.yaxis.set_tick_params(size=8)

    lo=hi=0

    eiglist = range(len(evals)) if eiglist is None else [i for i in eiglist if i < len(evals)]
    for i in eiglist:

        if asprobdens:
            ax2.plot(vecx, evecs[:,i]**2)
            lo=min(lo,(evecs[:,i]**2).min())
            hi=max(hi, (evecs[:,i]**2).max())
        else:
            ax2.plot(vecx, evecs[:,i])
            lo=min(lo, evecs[:,i].min())
            hi=max(hi, evecs[:,i].max())
    mhi=max(abs(lo), abs(hi))*1.1
    if asprobdens:
        ax2.set_ylim(0, mhi)
    else:
        ax2.set_ylim(-mhi, mhi)

def boundstates_plot_eigvecs_waterfall(soln, ax=None, eiglist=None, asprobdens=False, xlims=None, ylims=None):

    if ax is None: ax=plt.subplot(111)

    V=soln['Vx']
    evals=soln['evals']
    evecs=soln['evecs']

    vecx=soln['vecx']
    domain=[vecx[0], vecx[-1]]
    vp=np.array(list(map(V, vecx)))


    if soln['is_radial']:
      ax.set_xlabel('$r$')
      ax.set_ylabel(r'$\psi(r)$')
    else:
      ax.set_xlabel('$x$')
      ax.set_ylabel(r'$\psi(x)$')

    if xlims is None:
        ax.set_xlim(domain[0], domain[1])
    else:
        ax.set_xlim(xlims[0], xlims[1])

    lo=hi=0
    if asprobdens:
      lo0= (evecs[:,eiglist[0]]**2).min()
      hi1= (evecs[:,eiglist[-1]]**2).min()
    else:
      lo0=evecs[:,eiglist[0]].min()
      hi1= evecs[:,eiglist[-1]].min()

    for i in eiglist:
      if asprobdens:
        lo=min(lo,(evecs[:,i]**2).min())
        hi=max(hi, (evecs[:,i]**2).max())
      else:
        lo=min(lo, evecs[:,i].min())
        hi=max(hi, evecs[:,i].max())

    dy=(hi-lo)/len(eiglist)*3
    for i,e in enumerate(eiglist):
        if asprobdens:
          ax.plot(vecx, evecs[:,e]**2+i*dy)
        else:
          ax.plot(vecx, evecs[:,e]+i*dy)
    ax.set_ylim(lo0-dy, hi1+dy*len(eiglist)+dy)


def boundstates_plot_all(soln, asprobdens=0, eiglist=None, ylims=None, xlims=None):

    '''
    Display a grid of four key plots for potential well problems.

    The plots displayed are eigenenergies as a function of index, eigenenergies on top of the potential well,
    eigenenergies and eigenfunctions on top of the potential well, and eigenfunctions on top of the well.

    Parameters
    ----------
    asprobdens : boolean
       If True, causes display of probability densities rather than amplitudes.

    eiglist : list of integers < numeigs
       Specifies that only the listed states should be  displayed.
    '''

    fig=plt.figure(figsize=[12,8])
    axs = fig.subplots(2,2)
    fig.subplots_adjust(wspace=.3, hspace=.5)
    axs=axs.flat

    with mpl.rc_context({'xtick.labelsize':8,'font.size':6, 'axes.labelsize':10}):
        boundstates_plot_eigvals(soln, ax=axs[0], eiglist=eiglist)
        #boundstates_plot_eigvals_and_well(soln, ax=axs[1], eiglist=eiglist)
        boundstates_plot_eigvecs_waterfall(soln, ax=axs[1], eiglist=eiglist, asprobdens=asprobdens, xlims=xlims, ylims=ylims)
        boundstates_plot_eigvalsvecs_and_well(soln, ax=axs[2], eiglist=eiglist, xlims=xlims, ylims=ylims)
        boundstates_plot_eigvecs_and_well(soln, asprobdens=asprobdens, ax=axs[3], eiglist=eiglist, xlims=xlims, ylims=ylims)

def make_psi0(domain, xpts, shape, w0=1, x0=0, p0=0, mode=0):

    '''
    Create an initial wave function to seed a temporal evolution of the wave equation.

    This function determines the domain and grid spacing, and the form of the initial wave function, including functional shape,
    width, center position and momentum.

    Available functional forms are
        'gaussian' :   exp(-0.5* ((x-x0)/w0)**2) * exp(1j*p0*x)
        'supergaussian'   :   exp(-0.5* ((x-x0)/w0)**4) * exp(1j*p0*x)
        'hermitegaussian' :   H_n((x-x0)/w0)*exp(-0.5* ((x-x0)/w0)**2) * exp(1j*p0*x)
        'box'             :

    Parameters
    ----------
    domain    :  Tuple of floats (x0, x1)
                 Specifies the extent of the x domain.
    xpts      :  int
                 Number of grid points along the x-axis.
    shape     :  string
                 Choice of wave function shape from 'gaussian' or 'square'

    w0, x0, p0:  floats
                 Value of initial width, mean position and momentum

    Returns
    -------
    A dictionary containing the following elements:

       vecx    : A numpy float array of x-values
       delta_x : The spacing of x-values in the solution
       xpts:   : The number of x-values in the grid
       psi     : A numpy complex float array holding the initial wave function
    '''

    vx=np.linspace(domain[0], domain[1], xpts)
    dx=vx[1]-vx[0]
    if shape=='gaussian':
        psi=np.exp(-.5*((vx-x0)/w0)**2, dtype=complex)
    elif shape=='supergaussian':
        psi=np.exp(-.5*((vx-x0)/w0)**4, dtype=complex)
    elif shape=='hermitegaussian':
        carr=np.zeros(mode+1, dtype=int)
        carr[mode]=1
        psi=np.exp(-.5*((vx-x0)/w0)**2, dtype=complex)*herm.hermval((vx-x0)/w0, carr)
    elif shape=='box':
        L=vx[-1]-vx[0]
        psi=np.sin((vx-vx[0])*mode/L*pi, dtype=complex)

    else:
        print ('No such shape: %s'% shape)
        raise Error
    psi/=sqrt(np.sum(np.conj(psi)*psi).real*dx)
    psi*=np.exp(1j*p0*vx)
    return {'vecx':vx, 'delta_x':dx, 'xpts':len(vx), 'psi':psi}

def qerror(msg):
    print('Error: %s'%msg)
    return None

def make_psi_initial_state(eigsol, coeffs, shiftx=0, stretchx=1):
    evals=eigsol['evals']
    evecs=eigsol['evecs']
    vecx=eigsol['vecx']
    dx=vecx[1]-vecx[0]
    dim=len(evals)

    npc=np.array(coeffs)
    if np.real(np.dot(npc.conj(), npc)) < 1e-10:
        print('Empty initial coefficient specification ')
        return None
    psi_coeffs=np.zeros(dim, np.complex128)
    psi_coeffs[:len(coeffs)]=coeffs

    psi_coeffs=psi_coeffs/sqrt(np.real(np.dot(psi_coeffs.conj(), psi_coeffs)))

    if shiftx==0 and stretchx==1: return psi_coeffs

    psi_x0=evecs[:,0]*0
    for j in range(dim): psi_x0=psi_x0+evecs[:,j]*psi_coeffs[j]

    if stretchx!=1:
        psi_interp=scipy.interpolate.interp1d(vecx, psi_x0, bounds_error=False, fill_value=0.0)
        vecx_str=vecx / stretchx
        psi_x0=psi_interp(vecx_str)
    if shiftx!=0:
        psi_x0=np.roll(psi_x0, int(round(shiftx/dx)))

    psi_coeffs=psi_coeffs*0
    for j in range(dim):
        psi_coeffs[j]=scipy.integrate.simps(evecs[:,j].conj()*psi_x0, dx=dx)
    psi_coeffs=psi_coeffs/sqrt(np.real(np.dot(psi_coeffs.conj(), psi_coeffs)))

    return psi_coeffs

def evolve_by_stationary_state_expansion(eigsol, psi0, tmax, tsteps):
    xpts=len(eigsol['vecx'])
    evals=eigsol['evals']
    evecs=eigsol['evecs']
    vecx=eigsol['vecx']
    dx=vecx[1]-vecx[0]
    dim=len(evals)

    psi_coeffs=psi0

    dt=tmax/tsteps

    m_psi=np.zeros([tsteps+1, xpts], dtype=complex)

    tol=1e-10
    for j in range(dim):
        if abs(psi_coeffs[j])<tol: continue #eigenvector not involved
        dphase=np.exp(-1j*evals[j]*dt)
        phase=1.0
        for tj in range(tsteps+1):

            m_psi[tj,:]=m_psi[tj,:] + psi_coeffs[j]*phase * evecs[:,j]
            phase = phase * dphase

    sol=copy.deepcopy(eigsol)
    V=eigsol['Vx']
    vx=eigsol['vecx']
    vecVx=np.array(list(map(V, vx)))

    sol['psi']=m_psi
    sol['vect']=np.linspace(0,tmax,tsteps+1)
    sol['vecVx']=vecVx
    sol['tpts']=tsteps+1

    return sol

@control_animation
def animate_psi(psi_evol, skip=1, asprobdens=0, fig=None, ax=None):

    if fig is None:
        fig, ax = plt.subplots()
    vecx=psi_evol['vecx']
    m_psi=psi_evol['psi']

    vt=psi_evol['vect']
    dt=vt[1]-vt[0]
    vecVx=psi_evol['vecVx']
    nframes=m_psi.shape[0]
    psimax=np.abs(m_psi).max()
    ax.set_xlim(vecx[0], vecx[-1])
    if asprobdens:
        ax.set_ylim(0, psimax**2*1.05)
    else:
        ax.set_ylim(-psimax*1.05, psimax*1.05)
    ax.set_xlabel('$x$')
    ax.set_ylabel('$\psi(x,t), |\psi(x,t)|^2$')
    ax2=ax.twinx()
    ax2.set_xlim(vecx[0], vecx[-1])
    ax2.set_ylim(0, np.amax(vecVx))
    if vecVx.ndim == 1: ax2.plot(vecx, vecVx, 'gray')

    if asprobdens:
        l1, = ax.plot([], [], 'r-')
        l2 = None
        l3 = None
    else:
        if vecVx.ndim != 1: l4, = ax2.plot([], [], 'gray')
        l1, = ax.plot([], [], 'b-')
        l2, = ax.plot([], [], 'r:')
        l3, = ax.plot([], [], 'g:')

    if vecVx.ndim == 1:
      anim = animation.FuncAnimation(fig, __update_psi, nframes,
          fargs=(vecx, m_psi, l1, l2, l3, ax, dt, asprobdens), interval=100, blit=True)
    elif vecVx.ndim != 1:
      anim = animation.FuncAnimation(fig, __update_psi_potevol, nframes,
          fargs=(vecx, m_psi, vecVx, l1, l2, l3, l4, ax, dt, asprobdens), interval=100, blit=True)
    return anim

def __update_psi(i, vecx, m_psi, line1, line2, line3, ax, dt, asprobdens):
    if asprobdens:
        line1.set_data(vecx, np.abs(m_psi[i,:])**2 )
    else:
        line2.set_data(vecx, m_psi[i,:].real)
        line3.set_data(vecx, m_psi[i,:].imag)
        line1.set_data(vecx, np.abs(m_psi[i,:]))

    #ax.set_title('$\\tau=%.2f$'%(i*dt))
    return line1,

def __update_psi_potevol(i, vecx, m_psi, vecVx, line1, line2, line3, line4, ax, dt, asprobdens):
    if asprobdens:
        line1.set_data(vecx, np.abs(m_psi[i,:])**2 )
    else:
        line2.set_data(vecx, m_psi[i,:].real)
        line3.set_data(vecx, m_psi[i,:].imag)
        line1.set_data(vecx, np.abs(m_psi[i,:]))
        line4.set_data(vecx, vecVx[i,:].real)

    #ax.set_title('$\\tau=%.2f$'%(i*dt))
    return line1,

All the code below has been produced by Mr. Thomas Dinter (School of Mathematical & Physical Sciences, Macquarie University).

In [ ]:
%%capture

from matplotlib import rc
rc('animation', html='jshtml')
import matplotlib.animation as animation
!pip install qutip
from qutip import *
import scipy.fftpack as fftpack

def make_harmonic_potential(k = 1, q = 0):
    def Vfunc(x):
        return (1 / 2) * k * (x**2) + q * (x**3)
    return Vfunc

def make_morse_potential(d = 5, a = 0.25, c = 0):

    '''
    Produces a Morse potential function. Takes input parameters:
            d     :    The 'depth' of the potential well.
            a     :    Corresponds to the inverse-width of the well. Large a = narrow well.
            c     :    Centre of the well.
    '''

    def Vfunc(x):
        return d * (np.exp(-2 * a * (x - c)) - 2 * np.exp(-a * (x - c)))
    return Vfunc

def create_superposition(solution_dict, states = None, coefs = None):

    '''
    Generates superpositions for time-evolving. 'States' is an array of numbers 0 <= x <= max(eigvalue) in the solutions dictionary, 'coefs' are the coefficients of these states in a superposition,
    in the same order as they are listed in 'states'.
    '''

    if coefs == None:
        coefs = np.ones(len(states))

    if len(states) != len(coefs):
        print("Number of states and coefficients do not match.")
        return None

    coef_array = []
    num = 0
    for i in range(0, eigsols['numeigs']):
        if i in states:
            coef_array.append(coefs[num])
            num = num + 1
        else:
            coef_array.append(0)
    return coef_array

def eigfunc(n, xmax = 6, xpts = 1000):

    '''
    Returns a dictionary containing the following elements:
         psi    : wavefunction of specified eigenvalue
         domain : the domain over which the wavefunction is specified
    '''

    psi = []
    func = scipy.special.hermite(n)                   # Hermite polynomial
    A = sqrt(1 / (sqrt(pi) * (2**n) * factorial(n)))  # Normalization coefficient
    for i in np.linspace(-xmax, xmax, xpts):
        psi.append(A * exp(-(i**2) / 2) * func(i))
    return {'psi':psi, 'domain':np.linspace(-xmax, xmax, xpts)}

def crank_nicolson_method(delta_x = 0.1, delta_t = 1e-5, m = 1, vp = None):
    alpha = delta_t / (2 * m * delta_x**2)
    Mt = np.zeros((len(vp), len(vp)), dtype = np.complex128)
    off_diag = np.full(len(vp) - 1, 1j * alpha, dtype = np.complex128)
    Mt += np.diag(off_diag, 1)
    Mt += np.diag(off_diag, -1)
    Mtdt = np.copy(Mt)
    Mt += np.diag(1 - 2 * 1j * alpha - 1j * delta_t * vp)
    Mtdt += np.diag(1 - 2 * 1j * alpha - 1j * delta_t * vp)
    return 0.5 * np.linalg.inv(np.eye(len(vp), dtype = np.complex128) - 0.5 * Mtdt) @ Mt

def evolve_state_schro(eigsol, psi0, tmax, tsteps):
    xpts=len(eigsol['vecx'])
    evals=eigsol['evals']          # Array of eigenvalues.
    evecs=eigsol['evecs']          # Array of spatial wavefunctions.
    vecx=eigsol['vecx']            # Domain
    dim=len(evals)

    deltax = abs(vecx[1] - vecx[0]) # Spacing between grid points.
    dt = tmax / tsteps              # Time step
    psi_coeffs = psi0               # Extract initial wavefunction.

    evolved_psi = np.zeros([tsteps + 1, xpts], dtype = complex)   # Stores the actual solution
    potential_t = np.zeros([tsteps + 1, xpts], dtype = complex)   # Stores the potential at each time.
    potential_t[0, :] = make_harmonic_potential(1, 0)(vecx)       # Assume we always start with this potential.

    tol = 1e-10
    for j in range(dim):
        if abs(psi_coeffs[j]) < tol: continue                   # eigenvector not involved
        m_psi = np.zeros([tsteps + 1, xpts], dtype = complex)   # m_psi is a zero matrix. Each row is a time-step, and each column is a spatial point. Stores the temporary solution.
        m_psi[0,:] = psi_coeffs[j] * evecs[:,j]                 # Initialize the wavefunction at time zero.
        evolved_psi[0,:] = evolved_psi[0,:] + m_psi[0,:]

        for tj in range(1, tsteps + 1):
            previous_psi = m_psi[tj-1,:]
            potential_t[tj,:] += make_harmonic_potential(1 + 1 * tj * dt, 0)(vecx)          # POTENTIAL function, V(x,t). The next line adds auxillary dynamics if required.                                                                     # Store V(x,t) for plotting.
            C = crank_nicolson_method(deltax, dt, 1, potential_t[tj,:])                     # All the time evolution is contained in this function, defined above.
            m_psi[tj,:] = m_psi[tj,:] + C @ previous_psi
            evolved_psi[tj,:] = evolved_psi[tj,:] + m_psi[tj,:]

    sol=copy.deepcopy(eigsol)
    sol['psi']=evolved_psi
    sol['vect']=np.linspace(0,tmax,tsteps + 1)
    sol['vecVx']=potential_t
    sol['tpts']=tsteps + 1

    return sol

def check_num_acc(state_one, state_two, arr = False):

    '''
    Takes the time-evolution of two states and computes their "similarity". This similarity is the average of the absolute value of their inner-product, for all time t.
    '''

    wavefunc_one = state_one['psi']; wavefunc_two = state_two['psi']    # Extracts the wavefunction from the dictionary.
    if wavefunc_one.shape != wavefunc_two.shape:
        print('ERROR: Input states are not of the same dimension')
    return None

    sim = []
    wavefunc_one = np.conj(wavefunc_one)
    for i in range(0, wavefunc_one.shape[0]):
        sim.append(abs(np.trapz(wavefunc_one[i,:] * wavefunc_two[i,:]))**2)    # The inner-product of the two solutions at a particular time.
    sim = sim / sim[0]
    if arr == True: print(sim)
    plt.xlabel('Time [arb. units]')
    plt.ylabel(r'Similarity, $\int \psi_{\mathrm{exact}}^{*}(t)\psi_{\mathrm{approx.}}(t)~\mathrm{d}x$')
    #plt.ylim([min(sim), 1.])
    plt.plot(sim)

def harmonic_eigfunc(n=0):
    def psi(x):
        func=scipy.special.hermite(n)
        return sqrt(1 / (sqrt(pi) * (2**n) * factorial(n))) * np.exp(-x**2 / 2) * func(x)
    return psi    

def initial_fock(N=50, n=[0], coeffs=[1], extents=[-10,10,-10,10], pts=[200,200]):
    
    if len(n) != len(coeffs):
        print('Number of states and coefficients do not match')
        return None
    norm = 0
    for i in range(len(coeffs)):
        norm += coeffs[i]**2
    norm = sqrt(norm)    
    wavefunc = 0
    for i in range(len(coeffs)):
        wavefunc += coeffs[i] / norm * fock(N, n[i])
    Wfunc = wigner(wavefunc, np.linspace(extents[0], extents[1], pts[0]), np.linspace(extents[2], extents[3], pts[1]))
    
    return {'initial_func':Wfunc, 'extents':extents, 'numpts':pts}

def initial_coherent(N=50, alpha=3, extents=[-10,10,-10,10], pts=[200,200]):
    
    x_coherent = np.linspace(extents[0] - alpha, (extents[1] - alpha) * (1 - 2 / pts[0]), pts[0])
    Wfunc = wigner(fock(N, 0), x_coherent, np.linspace(extents[2], extents[3], pts[1]))
    
    return {'initial_func':Wfunc, 'extents':extents, 'numpts':pts}

def initial_thermal(N=50, nexp=0.5, extents=[-10,10,-10,10], pts=[200,200]):
    
    rho_thermal = thermal_dm(N, nexp)
    Wfunc = wigner(rho_thermal, np.linspace(extents[0], extents[1], pts[0]), np.linspace(extents[2], extents[3], pts[1]))
    
    return {'initial_func':Wfunc, 'extents':extents, 'numpts':pts}

def SHO_Wigner(initial_dict, pot=make_harmonic_potential(1,0), tmax=1, tsteps=100, gamma = 0, title=None):

    '''
    This is a function which constructs the Wigner function of some superposition of SHO eigenstates, and evolves it under an arbitrary potential.

    Returns a dictionary with the following elements:
        'Wigner_func'    :    Contains the Wigner function at each time-step.
        'extents'        :    The 'extent' of phase-space being simulated.
        'numpts'         :    An array of the form, [# spatial points, # momentum points]
        'title'          :    Title for displating the evolution. If a title is not specified as input, it is simply the initial wavefunction in ket notation.
        'vect'           :    Array of the time considered for evolution.
    '''
    
    extents = initial_dict['extents']
    pts = initial_dict['numpts']
    
    xpts = pts[0]; ppts = pts[1]
    x_vector = np.linspace(extents[0], extents[1] * (1 - 2 / xpts), xpts)
    p_vector = np.linspace(extents[2], extents[3] * (1 - 2 / ppts), ppts)
    theta_vector = fftpack.fftshift(2 * pi * fftpack.fftfreq(p_vector.size, p_vector[1] - p_vector[0]))   # Define theta and lambda as the conjugate variables of x & p.
    lambda_vector = fftpack.fftshift(2 * pi * fftpack.fftfreq(x_vector.size, x_vector[1] - x_vector[0]))

    X, Theta = np.meshgrid(x_vector, -theta_vector)
    Lambda, P = np.meshgrid(lambda_vector, -p_vector)

    dt=tmax / tsteps
    evolved_Wigner=np.zeros([tsteps + 1, xpts, ppts], dtype = np.complex128)       # Stores the Wigner function for all time.
    evolved_Wigner[0,:,:] = initial_dict['initial_func']

    '''
    We have now constructed the initial Wigner function, so the remainder of this function is concerned with it's evolution.
    '''

    potentialPropogatorFactor = fftpack.ifftshift(np.exp(-1j * dt * (pot(X - Theta / 2) - pot(X + Theta / 2)) 
                                                         - dt * gamma * Theta**2), axes = (0,))
    kineticPropogatorFactor = fftpack.ifftshift(np.exp(-1j * Lambda * P * dt / 2), axes = (1,))

#     pot_dx = np.gradient(pot(X), x_vector, axis=(1,))                                   # Define the gradient of the potential.
#     LFactorOne = fftpack.ifftshift(np.exp(1j * P * Lambda * dt / 2), axes = (1,))       # Define the kinetic part of the Liouvillian.
#     LFactorTwo = fftpack.ifftshift(np.exp(-1j * pot_dx * Theta * dt), axes = (0,))      # Define the potential part of the Liouvillian.

    for tj in range(1, tsteps + 1):                         # For each time-step ...

        ''' This part takes care of the unitary evolution under Moyal's equation. '''

        previous_Wigner = evolved_Wigner[tj - 1,:,:]
        W = fftpack.fft(previous_Wigner, axis = 1)            # Fourier transfrom from x --> lambda
        W *= kineticPropogatorFactor                          # Apply the kinetic time-evolution operator.
        W = fftpack.ifft(W, axis = 1)                         # Fourier transform from lambda --> x
        W = fftpack.fft(W, axis = 0)                          # Fourier transfrom from p --> theta
        W *= potentialPropogatorFactor                        # Apply the potential time-evolution operator.
        W = fftpack.fft(W, axis = 1)                          # Fourier transfrom from x --> lambda
        W = fftpack.ifft(W, axis = 0)                         #                    theta --> p
        W *= kineticPropogatorFactor                          # Apply the kinetic time-evolution operator.
        W = fftpack.ifft(W, axis = 1)                         # Fourier transform from lambda --> x

        ''' This part includes open quantum system dynamics. '''

#         W1 = P * W                                              # Multiply by p.
#         W1 = fftpack.fft(W1, axis = 0)                          # Fourier transfrom from p --> theta
#         W1 = fftpack.ifftshift(Theta, axes = (0,)) * W1         # Multiply by theta.
#         W1 = fftpack.ifft(W1, axis = 0)                         # Fourier transfrom from theta --> p
#         W_tot = W + 1j * dt * gamma * W1

#         W1 = P * W_tot
#         W1 = fftpack.fft(W1, axis = 0)
#         W1 = fftpack.ifftshift(Theta, axes = (0,)) * W1
#         W1 = fftpack.ifft(W1, axis = 0)
#         W1 = 2 * 1j * dt * gamma * W1

#         evolved_Wigner[tj,:,:] += W + W1
        evolved_Wigner[tj,:,:] += W

    if title==None:
        title=r'${:.2f}|{}\rangle$'.format(coeffs[0], n[0])
        for i in range(1, len(n)): title += r'$+{:.2f}|{}\rangle$'.format(coeffs[i], n[i])

    return {'Wigner_func':evolved_Wigner.real, 'extents':extents, 'numpts':pts, 'initial_func':evolved_Wigner[-1,:,:], 
          'title':title, 'vect':np.linspace(0,tmax,tsteps+1), 'grid':[X,P,Theta,Lambda]}

def animate_Wigner(Wigner_dict, view_lims=None, clims=[-1,1]):

    Wigner_func = Wigner_dict['Wigner_func']      # Contains the time-evolution of the Wigner function.
    extents = Wigner_dict['extents']              # Contains the domain info. about the plot.
    title = Wigner_dict['title']                  # Carries the title specified in the call of "SHO_Wigner".

    fig, ax = plt.subplots()
    ax.set_xlabel(r'$x~/~x_{zpf}$')
    ax.set_ylabel(r'$p~/~p_{zpf}$')
    ax.set_title(title)

    if view_lims != None:
        ax.set_xlim(view_lims[0], view_lims[1])
        ax.set_ylim(view_lims[2], view_lims[3])
    plot = ax.imshow(Wigner_func[0], extent=[extents[0], extents[1], extents[2], extents[3]], cmap='seismic', vmin=clims[0], vmax=clims[1], origin='lower');

    fig.colorbar(plot)
    anim = animation.FuncAnimation(fig, __update_Wigner, frames=Wigner_func.shape[0]-1, fargs=(Wigner_func, plot, extents, title), blit=False, repeat=True);
    
    return anim

def __update_Wigner(w, Wigner_func, plot, extents, title):
    plot.set_data(Wigner_func[w]);
    return plot,
    
def check_statistics(input_dict):
    X,P,Theta,Lambda = input_dict['grid']
    cell_size = (input_dict['extents'][1] - input_dict['extents'][0]) / input_dict['numpts'][0] * (input_dict['extents'][3] - input_dict['extents'][2]) / input_dict['numpts'][1]
    
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(7, 5), tight_layout = True, sharex = True)
    
    position_sq = []
    for t in range(len(input_dict['vect'])):
        current_func = input_dict['Wigner_func'][t]
        total = sum(map(sum, np.multiply(X**2, current_func)))
        position_sq.append(cell_size * total)
#     plt.plot(input_dict['vect'], position_sq, color = 'red', label = r'$\langle x^{2} \rangle$')
    
    position = []
    for t in range(len(input_dict['vect'])):
        current_func = input_dict['Wigner_func'][t]
        total = sum(map(sum, np.multiply(X, current_func)))
        position.append(cell_size * total)
#     plt.scatter(input_dict['vect'], position, color = 'green', label = r'$\langle x \rangle$')

    momentum_sq = []
    for t in range(len(input_dict['vect'])):
        current_func = input_dict['Wigner_func'][t]
        total = sum(map(sum, np.multiply(P**2, current_func)))
        momentum_sq.append(cell_size * total)
#     plt.plot(input_dict['vect'], position_sq, color = 'red', label = r'$\langle x^{2} \rangle$')
    
    momentum = []
    for t in range(len(input_dict['vect'])):
        current_func = input_dict['Wigner_func'][t]
        total = sum(map(sum, np.multiply(X, current_func)))
        momentum.append(cell_size * total)
#     plt.scatter(input_dict['vect'], position, color = 'purple', label = r'$\langle x \rangle$')
        
    Δx = []
    for i in range(len(position)):
        Δx.append(sqrt(position_sq[i] - (position[i])**2))
        
    Δp = []
    for i in range(len(momentum)):
        Δp.append(sqrt(momentum_sq[i] - (momentum[i])**2))

    purity = []
    for t in range(len(input_dict['vect'])):
        current_func = input_dict['Wigner_func'][t]
        total = sum(map(sum, np.multiply(current_func, current_func)))
        purity.append(cell_size * 2 * np.pi * total)
#     plt.scatter(input_dict['vect'], purity, color = 'black', label = r'Tr($\rho^{2}$)')
        
    colen = []
    for i in range(len(position)):
        colen.append(sqrt(8) * purity[i] * Δx[i])
        
    ax1.plot(input_dict['vect'], Δx, color = 'red', linewidth = 3, label = r'$\Delta q(\tau)~/~q_{\mathrm{zpf}}$')
    ax1.plot(input_dict['vect'], Δp, color = 'blue', linewidth = 3, label = r'$\Delta p(\tau)~/~p_{\mathrm{zpf}}$')
    ax1.axhline(1/sqrt(2), color = 'black', linestyle='dashed') #, label=r'$\sqrt{\hbar~/~2}~/~q_{\mathrm{zpf}}$')
    ax1.set_xlim(0, np.max(input_dict['vect']))
    ax1.set_ylim(0, 1.1 * np.max(Δx))
    ax1.set_ylabel(r'Uncertainties')
    ax1.legend(loc = 'best')
    
    ax2.plot(input_dict['vect'], purity, color = 'black', linewidth = 3, label = r'Tr$[\rho(\tau)^{2}]$')
    ax2.set_ylim(0, 1)
    ax2.set_ylabel(r'State Purity')
    ax2.legend(loc = 'best')
    
    ax3.plot(input_dict['vect'], colen, color = 'red', linewidth = 3, label = r'$\xi(\tau)~/~q_{\mathrm{zpf}}$')
    ax3.set_xlabel(r'$\tau$')
    ax3.set_ylim(0, 1.1 * np.max(colen))
    ax3.axhline(colen[0], linestyle = '--', color = 'black')
    ax3.axhline(colen[-1], linestyle = '--', color = 'black')
    ax3.set_ylabel(r'Coherence length')
    ax3.legend(loc = 'best')
    plt.show()
    
    print('Increase in coherence length: {}'.format(colen[-1] - colen[0]))
    print('Final Position Uncertainty: {}'.format(Δx[-1]))
    print('Final Momentum Uncertainty: {}'.format(Δp[-1]))
    print('Uncertainty Principle Satisfied: {}'.format(Δx[-1]*Δp[-1] >= 1/2))
    
def uncertainty_vcoupling(input_dict):
    X,P,Theta,Lambda = input_dict['grid']
    cell_size = (input_dict['extents'][1] - input_dict['extents'][0]) / input_dict['numpts'][0] * (input_dict['extents'][3] - input_dict['extents'][2]) / input_dict['numpts'][1]
    
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(7, 5), tight_layout = True, sharex = True)
    
    position_sq = []
    for t in range(len(input_dict['vect'])):
        current_func = input_dict['Wigner_func'][t]
        total = sum(map(sum, np.multiply(X**2, current_func)))
        position_sq.append(cell_size * total)
#     plt.plot(input_dict['vect'], position_sq, color = 'red', label = r'$\langle x^{2} \rangle$')
    
    position = []
    for t in range(len(input_dict['vect'])):
        current_func = input_dict['Wigner_func'][t]
        total = sum(map(sum, np.multiply(X, current_func)))
        position.append(cell_size * total)
#     plt.scatter(input_dict['vect'], position, color = 'green', label = r'$\langle x \rangle$')

    momentum_sq = []
    for t in range(len(input_dict['vect'])):
        current_func = input_dict['Wigner_func'][t]
        total = sum(map(sum, np.multiply(P**2, current_func)))
        momentum_sq.append(cell_size * total)
#     plt.plot(input_dict['vect'], position_sq, color = 'red', label = r'$\langle x^{2} \rangle$')
    
    momentum = []
    for t in range(len(input_dict['vect'])):
        current_func = input_dict['Wigner_func'][t]
        total = sum(map(sum, np.multiply(X, current_func)))
        momentum.append(cell_size * total)
#     plt.scatter(input_dict['vect'], position, color = 'purple', label = r'$\langle x \rangle$')
        
    Δx = []
    for i in range(len(position)):
        Δx.append(sqrt(position_sq[i] - (position[i])**2))
        
    Δp = []
    for i in range(len(momentum)):
        Δp.append(sqrt(momentum_sq[i] - (momentum[i])**2))

    purity = []
    for t in range(len(input_dict['vect'])):
        current_func = input_dict['Wigner_func'][t]
        total = sum(map(sum, np.multiply(current_func, current_func)))
        purity.append(cell_size * 2 * np.pi * total)
#     plt.scatter(input_dict['vect'], purity, color = 'black', label = r'Tr($\rho^{2}$)')
        
    colen = []
    for i in range(len(position)):
        colen.append(sqrt(8) * purity[i] * Δx[i])
        
    ax1.set_title(input_dict['title'])
    ax1.scatter(input_dict['vect'], Δx, color = 'red', linewidth = 3, label = r'$\Delta q(\tau_{\mathrm{final}})~/~q_{\mathrm{zpf}}$')
    ax1.scatter(input_dict['vect'], Δp, color = 'blue', linewidth = 3, label = r'$\Delta p(\tau_{\mathrm{final}})~/~p_{\mathrm{zpf}}$')
    ax1.axhline(1/sqrt(2), color = 'black', linestyle='dashed') #, label=r'$\sqrt{\hbar~/~2}~/~q_{\mathrm{zpf}}$')
    ax1.set_xlim(0, np.max(input_dict['vect']))
    ax1.set_ylim(0, 1.1 * np.max(Δx))
    ax1.set_ylabel(r'Uncertainties')
    ax1.legend(loc = 'best')
    
    ax2.scatter(input_dict['vect'], purity, color = 'black', linewidth = 3, label = r'Tr$[\rho(\tau_{\mathrm{final}})^{2}]$')
    ax2.set_ylim(0, 1)
    ax2.set_ylabel(r'State Purity')
    ax2.legend(loc = 'best')
    
    ax3.scatter(input_dict['vect'], colen, color = 'red', linewidth = 3, label = r'$\xi(\tau_{\mathrm{final}})~/~q_{\mathrm{zpf}}$')
    ax3.set_xlabel(r'Decoherence rate ($\gamma$)')
    ax3.set_ylim(0, 1.1 * np.max(colen))
    ax3.set_ylabel(r'Coherence length')
    ax3.legend(loc = 'best')
    
    def coherence_fit(x, A, B, C, D):
        return A * np.exp(-B * x + C) + D
    
    gamma_fit = np.linspace(0, np.max(input_dict['vect']), 100)
    initial_parameters = [1, 1, 0, 0]
    params, params_cov = op.curve_fit(coherence_fit, input_dict['vect'], colen / np.max(colen), p0 = initial_parameters)
    ax3.plot(gamma_fit, np.max(colen) * coherence_fit(gamma_fit, params[0], params[1], params[2], params[3]),
            linestyle = '--', color = 'black', linewidth = 3)
    plt.show()
    print('Decay rate: {}'.format(params[1]))

We are interested in numerically modelling the time-evolution of the so-called _Wigner function_ for an open quantum system. In particular, the time-evolution of functions in phase-space is governed by _Moyal's equation_, of the form

$$
    i\hbar\frac{\partial}{\partial t} W(\hat{\boldsymbol{x}}, \hat{\boldsymbol{p}}, t) = \Big\{\Big\{ W(\hat{\boldsymbol{x}}, \hat{\boldsymbol{p}}, t), \mathcal{H}(\hat{\boldsymbol{x}}, \hat{\boldsymbol{p}}, t) \Big\}\Big\} ~ ,
$$

where $\big\{\big\{a(\hat{\boldsymbol{x}}, \hat{\boldsymbol{p}}), b(\hat{\boldsymbol{x}}, \hat{\boldsymbol{p}})\big\}\big\} = a(\hat{\boldsymbol{x}}, \hat{\boldsymbol{p}}) \star_{\hbar} b(\hat{\boldsymbol{x}}, \hat{\boldsymbol{p}}) - b(\hat{\boldsymbol{x}}, \hat{\boldsymbol{p}}) \star_{\hbar} a(\hat{\boldsymbol{x}}, \hat{\boldsymbol{p}})$ refers to the _Moyal bracket_ and $\hat{\boldsymbol{x}}$ and $\hat{\boldsymbol{p}}$ refer to the canonical position and momentum, respectively. Here, we solve this equation using by introducing the four-operator algebra given by the bopp operators, $\hat{x}, \hat{p}, \hat{\theta}$ and $\hat{\lambda}$. After some math (see Thomas' Overleaf), Moyal's equation reduces to an equation of motion for the Wigner function of the form

$$
    i\hbar\frac{\partial}{\partial t}W_{xp\theta\lambda} = \Bigg[\frac{\hbar}{m}\hat{p}\hat{\lambda} + V\Bigg(\hat{x} - \frac{\hbar}{2}\hat{\theta}\Bigg) - V\Bigg(\hat{x}+\frac{\hbar}{2}\hat{\theta}\Bigg)\Bigg]W_{xp\theta\lambda} ~ ,
$$

where $W_{xp\theta\lambda}$ refers to a particular element of the Wigner function. We use this simplified EOM to propagate the Wigner function in a _spectral split-operator method_ (once again, for more details, see Thomas' Overleaf). This EOM can be easily extended to include open quantum system effects, as has been done here. In this case, the EOM above becomes

$$
    i\hbar\frac{\partial}{\partial t}W_{xp\theta\lambda} = \Bigg[\frac{\hbar}{m}\hat{p}\hat{\lambda} + V\Bigg(\hat{x} - \frac{\hbar}{2}\hat{\theta}\Bigg) - V\Bigg(\hat{x}+\frac{\hbar}{2}\hat{\theta}\Bigg) - \gamma\Big(2i\hat{\theta}\hat{p} + m\hbar\Omega\hat{\theta}{}^{2}\Big)\Bigg]W_{xp\theta\lambda} ~ ,
$$

where the $2i\gamma\hat{\theta}\hat{p}$ term and the $m\gamma\hbar\Omega\hat{\theta}{}^{2}$ term are responsible for dissipation and decoherence, respectively. These come from the _Caldeira-Legget_ master equation, commonly used to describe the damping and thermalization of a quantum harmonic oscillator. Specifically, the particular form used here assumes one is operating in the weak-coupling, high-temperature limit of Brownian motion. Sensible values for $\gamma$ are on the order of $0.1$ - values $\gamma\gg0.1$ are numerically unstable.

For the code implemented here, we assume that:

$$
    \hbar = 1 \\
     m = 1 \\
$$

and the Wigner functions generated by the function "initial_func(...)" are eigenstates of the $k=1$ harmonic potential well. This implies an oscillation period of $\tau = 2\pi$, which is nice and convenient. Additionally, for plotting purposes, both the position and momentum axes of phase-space have been scaled by the zero-point motion of the ground state

$$
    x_{\mathrm{zpf}} = \sqrt{\frac{\hbar}{2m\Omega}} \\
    p_{\mathrm{zpf}} = \sqrt{\frac{\hbar m\Omega}{2}} \\
$$

both of which simply reduce to $1~/~\sqrt{2}$ in this case.

Here we simply model the unitary dynamics of a coherent state $|\alpha\rangle$. Nothing particularly interesting happens ...

In [ ]:
Wigner_coherent=SHO_Wigner(initial_coherent(50, 4, extents=[-10,10,-10,10], pts=[200,200]),
                           pot = make_harmonic_potential(1, 0), tmax=4*pi, tsteps=100, gamma=0, title=r'$|\alpha\rangle$')
animate_Wigner(Wigner_coherent,[-10,10,-10,10], [-0.1, 0.1])

# Frequency Switching Approach

What might be more interesting is to investigate different method for _coherent expansion_ of the ground state. This is a process by which the coherence length of the ground state wavefunction is extended well beyond its usual limits, and is of great interest for performing matter-wave interferometry experiments with macroscopic objects. Most of the protocols for performing coherent expansion of the wavefunction rely on implementing pulse-like sequences for modulating the potential landscape seen by a levitated nanoparticle in an optical trap.

In particular, we implement the method proposed by M. Rossi _et al_., _"Quantum Delocalization of a Levitated Nanoparticle"_. (2024), available here (https://arxiv.org/pdf/2408.01264). In this approach, after having cooled the centre-of-mass motion of a levitated nanoparticle into a low-occupation thermal state (functionally equivalent to the motional quantum ground state), they expand the wavefunction by periodically modulating the optical tweezer power with a square-wave generator in a 2-pulse sequence. These pulses lower the mechanical resonance frequency to $\Omega_{\ell} = \Omega_{0}~/~r$, where $r$ denotes the frequency ratio. Their duration is $\pi~/~(2\Omega_{\ell})$, i.e., a quarter of the lower mechanical period. During both pulses, the position uncertainty increases proportionally to the frequency ratio $r$. The pulses are spaced apart by a quarter of the original period, i.e., $\pi~/~(2\Omega_{0})$, introduced to flip the sign of the average momentum that the nanoparticle has gained during the first pulse, e.g., due to stray electric fields.

This approach is implemented in the cell below, where we can follow the evolution of the Wigner function in phase-space. Following that, we explicitly calculate the position and momentum uncertainties through time ($\tau$). As a quick common-sense check we ensure that, for each time step, the _Uncertainty Principle_ is satisfied:

$$
    \Delta \hat{\boldsymbol{x}} \Delta \hat{\boldsymbol{p}} \geq \frac{\hbar}{2} ~ .
$$

Also, for completeness, it's worth acknowledging a few things.

**(1)** At the end of the protocol (after the 2nd bout in a reduced potential), one would typically switch to the standard (stiff) potential and begin measuring the nanoparticle's motion. The goal here is to estimate the nanoparticle's position and velocity at the moment you switch back to the standard (stiff) potential, to begin building statistical knowledge on the system in that moment. To conduct another trial, one would feedback cool the expanded state back to the motional quantum ground state, and begin anew.

**(2)** Even though large values of $r$ generates more squeezing, nanoparticles sitting in weak potentials are more sensitive to stray forces / fields in the environment. This is, for instance, why a 2-pulse sequence is used; to try and combat these effects. To quote E. Bonvin _et al_., _"State Expansion of a Levitated Nanoparticle in a Dark Harmonic Potential"_. (2024) (available here [https://doi.org/10.1103/PhysRevLett.132.253602], a paper that implements a 1-pulse frequency-switching approach):

_"Regarding the current limitations of our system, larger
 expansions could, in principle, be achieved with the help of
 weaker potentials. However, the associated increase in
 sensitivity to stray fields leads to frequent failures to
 recapture the particle in the optical trap and must be
 overcome with more accurate stray field and gravity
 compensation in the future."_

In [ ]:
r = 2
klow = 1 / r**2

potentials = [make_harmonic_potential(klow, 0), 
              make_harmonic_potential(1, 0), make_harmonic_potential(klow, 0)]
durations = [(2 * pi / sqrt(klow)) / 4, 2 * pi / 4, (2 * pi / sqrt(klow)) / 4]

initial_dict = initial_thermal(50, 0.5, extents=[-15,15,-15,15], pts=[200,200])

for i in range(len(potentials)):
    evolved_dict = SHO_Wigner(initial_dict, potentials[i], durations[i], tsteps=50,
                          gamma=0, title=r'2-Pulse Frequency Switching Approach')
#     plt.imshow(evolved_dict['Wigner_func'][-1]); plt.show()
    if i == 0:
        total_Wigner = evolved_dict['Wigner_func']
    if i != 0:
        total_Wigner = np.vstack([
            total_Wigner,
            evolved_dict['Wigner_func'],
        ])
    initial_dict = evolved_dict
    
total_dict = {'Wigner_func':total_Wigner, 'extents':initial_dict['extents'], 'numpts':initial_dict['numpts'], 
              'title':initial_dict['title'], 'vect':np.linspace(0,np.sum(durations),50*len(durations)), 'grid':initial_dict['grid']}
animate_Wigner(total_dict, [-15,15,-15,15], [-0.1, 0.1])

In [ ]:
check_statistics(total_dict)

# Catch & Release Approach

What might be surprising, is that the 2-pulse sequence explored above is actually just a means of approximating the _ideal_ approach to coherently expanding a levitated nanoparticle's wavefunction, which is to drop it. More specifically, people are interested in exploring the idea of, after having cooled a levitated nanoparticle to the motional quantum ground state, turning off the optical tweezer and allowing the nanoparticle to experience free-fall. During this free-fall, the wavefunction's position uncertainty ($\Delta \hat{\boldsymbol{x}}$) is expected to increase linearly as a function of time ($\tau$), while the momentum uncertainty ($\Delta \hat{\boldsymbol{p}}$) remains unaffected. Another key benefit of this approach is that the shot-noise in the system is greatly reduced (for obvious reasons; you turned the laser off). However, this approach is inevitably more technically challenging, as it requires re-capturing the nanoparticle to measure it's delocalized state.

In [ ]:
potentials = [make_harmonic_potential(0, 0)]
durations = [2 * pi / 1.5]

initial_dict = initial_thermal(50, 0.5, extents=[-15,15,-15,15], pts=[200,200])

for i in range(len(potentials)):
    evolved_dict = SHO_Wigner(initial_dict, potentials[i], durations[i], tsteps=50,
                          gamma=0, title=r'Free-fall Approach')
#     plt.imshow(evolved_dict['Wigner_func'][-1]); plt.show()
    if i == 0:
        total_Wigner = evolved_dict['Wigner_func']
    if i != 0:
        total_Wigner = np.vstack([
            total_Wigner,
            evolved_dict['Wigner_func'],
        ])
    initial_dict = evolved_dict
    
total_dict = {'Wigner_func':total_Wigner, 'extents':initial_dict['extents'], 'numpts':initial_dict['numpts'], 
              'title':initial_dict['title'], 'vect':np.linspace(0,np.sum(durations),50*len(durations)), 'grid':initial_dict['grid']}
animate_Wigner(total_dict, [-15,15,-15,15], [-0.1, 0.1])

In [ ]:
check_statistics(total_dict)

# Inverted Potential Approach

Another approach (in fact, the _optimal_ approach according to https://doi.org/10.1103/PhysRevLett.127.023601) to generating motional squeezing is to make use of an inverted harmonic potential. This approach will generate squeezing in the position quadrature of the wavefunction exponentially fast, but is difficult to implement experimentally and so has only (?) been explored in theory proposals. One option is to (potentially?) use a planar Paul trap. A more realistic choice might be to use a Morse potential, or to confine a nanoparticle to the centre of a double-well potential (https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.132.023601). The latter could be achieved, for instance, by generating a potential landscape with a Paul trap which weakly peturbs the (approximately) harmonic potential of an optical tweezer.

In [ ]:
potentials = [make_harmonic_potential(-1, 0), make_harmonic_potential(1, 0)]
durations = [2 * pi / 4, 2 * pi / 8]

initial_dict = initial_thermal(50, 0.5, extents=[-15,15,-15,15], pts=[200,200])

for i in range(len(potentials)):
    evolved_dict = SHO_Wigner(initial_dict, potentials[i], durations[i], tsteps=50,
                          gamma=0, title=r'Inverted Potential Approach')
#     plt.imshow(evolved_dict['Wigner_func'][-1]); plt.show()
    if i == 0:
        total_Wigner = evolved_dict['Wigner_func']
    if i != 0:
        total_Wigner = np.vstack([
            total_Wigner,
            evolved_dict['Wigner_func'],
        ])
    initial_dict = evolved_dict
    
total_dict = {'Wigner_func':total_Wigner, 'extents':initial_dict['extents'], 'numpts':initial_dict['numpts'], 
              'title':initial_dict['title'], 'vect':np.linspace(0,np.sum(durations),50*len(durations)), 'grid':initial_dict['grid']}
animate_Wigner(total_dict, [-15,15,-15,15], [-0.1, 0.1])

In [ ]:
check_statistics(total_dict)

# Open System Effects

In [ ]:
def uncertainty_vcoupling(input_dict):
    X,P,Theta,Lambda = input_dict['grid']
    cell_size = (input_dict['extents'][1] - input_dict['extents'][0]) / input_dict['numpts'][0] * (input_dict['extents'][3] - input_dict['extents'][2]) / input_dict['numpts'][1]
    
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(7, 5), tight_layout = True, sharex = True)
    
    position_sq = []
    for t in range(len(input_dict['vect'])):
        current_func = input_dict['Wigner_func'][t]
        total = sum(map(sum, np.multiply(X**2, current_func)))
        position_sq.append(cell_size * total)
#     plt.plot(input_dict['vect'], position_sq, color = 'red', label = r'$\langle x^{2} \rangle$')
    
    position = []
    for t in range(len(input_dict['vect'])):
        current_func = input_dict['Wigner_func'][t]
        total = sum(map(sum, np.multiply(X, current_func)))
        position.append(cell_size * total)
#     plt.scatter(input_dict['vect'], position, color = 'green', label = r'$\langle x \rangle$')

    momentum_sq = []
    for t in range(len(input_dict['vect'])):
        current_func = input_dict['Wigner_func'][t]
        total = sum(map(sum, np.multiply(P**2, current_func)))
        momentum_sq.append(cell_size * total)
#     plt.plot(input_dict['vect'], position_sq, color = 'red', label = r'$\langle x^{2} \rangle$')
    
    momentum = []
    for t in range(len(input_dict['vect'])):
        current_func = input_dict['Wigner_func'][t]
        total = sum(map(sum, np.multiply(X, current_func)))
        momentum.append(cell_size * total)
#     plt.scatter(input_dict['vect'], position, color = 'purple', label = r'$\langle x \rangle$')
        
    Δx = []
    for i in range(len(position)):
        Δx.append(sqrt(abs(position_sq[i] - (position[i])**2)))
        
    Δp = []
    for i in range(len(momentum)):
        Δp.append(sqrt(abs(momentum_sq[i] - (momentum[i])**2)))

    purity = []
    for t in range(len(input_dict['vect'])):
        current_func = input_dict['Wigner_func'][t]
        total = sum(map(sum, np.multiply(current_func, current_func)))
        purity.append(cell_size * 2 * np.pi * total)
#     plt.scatter(input_dict['vect'], purity, color = 'black', label = r'Tr($\rho^{2}$)')
        
    colen = []
    for i in range(len(position)):
        colen.append(sqrt(8) * purity[i] * Δx[i])
        
    ax1.set_title(input_dict['title'])
    ax1.scatter(input_dict['vect'], Δx, color = 'red', linewidth = 3, label = r'$\Delta q(\tau_{\mathrm{final}})~/~q_{\mathrm{zpf}}$')
    ax1.scatter(input_dict['vect'], Δp, color = 'blue', linewidth = 3, label = r'$\Delta p(\tau_{\mathrm{final}})~/~p_{\mathrm{zpf}}$')
    ax1.axhline(1/sqrt(2), color = 'black', linestyle='dashed') #, label=r'$\sqrt{\hbar~/~2}~/~q_{\mathrm{zpf}}$')
    ax1.set_xlim(1, np.max(input_dict['vect']))
    ax1.set_ylim(0, 1.1 * np.max(Δx))
    ax1.set_ylabel(r'Uncertainties')
    ax1.legend(loc = 'best')
    
    ax2.scatter(input_dict['vect'], purity, color = 'black', linewidth = 3, label = r'Tr$[\rho(\tau_{\mathrm{final}})^{2}]$')
    ax2.set_ylim(0, 1)
    ax2.set_ylabel(r'State Purity')
    ax2.legend(loc = 'best')
    
    ax3.scatter(input_dict['vect'], colen, color = 'red', linewidth = 3, label = r'$\xi(\tau_{\mathrm{final}})~/~q_{\mathrm{zpf}}$')
    ax3.set_xlabel(r'Frequency Ratio ($r$)')
    ax3.set_ylim(0, 1.1 * np.max(colen))
    ax3.set_ylabel(r'Coherence length')
    ax3.legend(loc = 'best')
    
    print(purity)
    print('/n')
    print(colen)
    
    plt.show()

In [ ]:
r = np.linspace(1, 4, 10)
gamma_val = 0.0001

final_funcs = []

for m in range(len(r)):
    
    klow = 1 / r[m]**2
    potentials = [make_harmonic_potential(klow, 0), 
              make_harmonic_potential(1, 0), make_harmonic_potential(klow, 0)]
    durations = [(2 * pi / sqrt(klow)) / 4, 2 * pi / 4, (2 * pi / sqrt(klow)) / 4]
    
    initial_dict = initial_thermal(50, 0.5, extents=[-40,40,-40,40], pts=[500,500])
    
    for i in range(len(potentials)):
        evolved_dict = SHO_Wigner(initial_dict, potentials[i], durations[i], tsteps=200,
                              gamma=gamma_val, title=r'Frequency Switching Method')
#         plt.imshow(evolved_dict['Wigner_func'][-1]); plt.show()
        if i == 0:
            total_Wigner = evolved_dict['Wigner_func']
        if i != 0:
            total_Wigner = np.vstack([
                total_Wigner,
                evolved_dict['Wigner_func'],
            ])
        initial_dict = evolved_dict
    
    final_funcs.append(total_Wigner[-1])
    
total_dict = {'Wigner_func':final_funcs, 'extents':initial_dict['extents'], 'numpts':initial_dict['numpts'], 
              'title':r'$\gamma$ = {}'.format(gamma_val), 'vect':r, 'grid':initial_dict['grid']}
uncertainty_vcoupling(total_dict)

# Several at a time ...

In [ ]:
def uncertainty_vcoupling(input_dict):
    X,P,Theta,Lambda = input_dict['grid']
    cell_size = (input_dict['extents'][1] - input_dict['extents'][0]) / input_dict['numpts'][0] * (input_dict['extents'][3] - input_dict['extents'][2]) / input_dict['numpts'][1]
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 5), tight_layout = True, sharex = True)
    colors = ['red', 'blue', 'green']
    
    for l in range(len(input_dict['gamval'])):
    
        position_sq = []
        for t in range(len(input_dict['vect'])):
            current_func = input_dict['Wigner_func'][l,t,:,:]
            total = sum(map(sum, np.multiply(X**2, current_func)))
            position_sq.append(cell_size * total)
    #     plt.plot(input_dict['vect'], position_sq, color = 'red', label = r'$\langle x^{2} \rangle$')

        position = []
        for t in range(len(input_dict['vect'])):
            current_func = input_dict['Wigner_func'][l,t,:,:]
            total = sum(map(sum, np.multiply(X, current_func)))
            position.append(cell_size * total)
    #     plt.scatter(input_dict['vect'], position, color = 'green', label = r'$\langle x \rangle$')

        momentum_sq = []
        for t in range(len(input_dict['vect'])):
            current_func = input_dict['Wigner_func'][l,t,:,:]
            total = sum(map(sum, np.multiply(P**2, current_func)))
            momentum_sq.append(cell_size * total)
    #     plt.plot(input_dict['vect'], position_sq, color = 'red', label = r'$\langle x^{2} \rangle$')

        momentum = []
        for t in range(len(input_dict['vect'])):
            current_func = input_dict['Wigner_func'][l,t,:,:]
            total = sum(map(sum, np.multiply(X, current_func)))
            momentum.append(cell_size * total)
    #     plt.scatter(input_dict['vect'], position, color = 'purple', label = r'$\langle x \rangle$')

        Δx = []
        for i in range(len(position)):
            Δx.append(sqrt(abs(position_sq[i] - (position[i])**2)))

        Δp = []
        for i in range(len(momentum)):
            Δp.append(sqrt(abs(momentum_sq[i] - (momentum[i])**2)))

        purity = []
        for t in range(len(input_dict['vect'])):
            current_func = input_dict['Wigner_func'][l,t,:,:]
            total = sum(map(sum, np.multiply(current_func, current_func)))
            purity.append(cell_size * 2 * np.pi * total)
    #     plt.scatter(input_dict['vect'], purity, color = 'black', label = r'Tr($\rho^{2}$)')

        colen = []
        for i in range(len(position)):
            colen.append(sqrt(8) * purity[i] * Δx[i])

        ax1.scatter(input_dict['vect'], purity, color = colors[l], linewidth = 3, label = r'$\gamma$ = {}'.format(input_dict['gamval'][l]))
        ax2.scatter(input_dict['vect'], colen, color = colors[l], linewidth = 3, label = r'$\gamma$ = {}'.format(input_dict['gamval'][l]))
        
    ax1.set_xlim(1, np.max(input_dict['vect']))
    ax1.set_ylim(0, 1)
    ax1.set_ylabel(r'State Purity')
    ax1.legend(loc = 'best')
    
    ax2.set_xlabel(r'Frequency Ratio ($r$)')
    ax2.set_ylim(0, 1.1 * np.max(colen))
    ax2.set_ylabel(r'Coherence length')
    ax2.legend(loc = 'best')
        
    plt.show()

In [ ]:
r = np.linspace(1, 4, 10)
gamma_val = [0.1, 0.01, 0.001]

final_funcs = np.zeros([len(gamma_val), len(r), 500, 500])

for l in range(len(gamma_val)):
    for m in range(len(r)):

        klow = 1 / r[m]**2
        potentials = [make_harmonic_potential(klow, 0), 
                  make_harmonic_potential(1, 0), make_harmonic_potential(klow, 0)]
        durations = [(2 * pi / sqrt(klow)) / 4, 2 * pi / 4, (2 * pi / sqrt(klow)) / 4]

        initial_dict = initial_thermal(50, 0.5, extents=[-40,40,-40,40], pts=[500,500])

        for i in range(len(potentials)):
            evolved_dict = SHO_Wigner(initial_dict, potentials[i], durations[i], tsteps=200,
                                  gamma=gamma_val[l], title=r'Frequency Switching Method')
    #         plt.imshow(evolved_dict['Wigner_func'][-1]); plt.show()
            if i == 0:
                total_Wigner = evolved_dict['Wigner_func']
            if i != 0:
                total_Wigner = np.vstack([
                    total_Wigner,
                    evolved_dict['Wigner_func'],
                ])
            initial_dict = evolved_dict

        final_funcs[l,m,:,:] += total_Wigner[-1]
    
total_dict = {'Wigner_func':final_funcs, 'extents':initial_dict['extents'], 'numpts':initial_dict['numpts'], 
              'vect':r, 'gamval':gamma_val, 'grid':initial_dict['grid']}
uncertainty_vcoupling(total_dict)